In [2]:
# --- Setup the Jupyter version of Dash ---
from jupyter_dash import JupyterDash
JupyterDash.infer_jupyter_proxy_config()

# --- Dashboard components ---
import dash  # needed for dash.callback_context (ctx in newer Dash)
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import base64, os

# --- Plotting & data ---
import pandas as pd
import plotly.express as px

# --- CRUD (use our module) ---
from aac_crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Use the creds that work on your Codio box
username = "aacuser"
password = "SNHU1234"   # if this fails, try "aacuser" and/or auth_source='admin'

db = AnimalShelter(username, password, auth_source="aac")

# Rescue → preferred breeds (simplified; adjust to match your spec doc if needed)
RESCUE_BREEDS = {
    "Water Rescue": [
        "Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland", "Portuguese Water Dog"
    ],
    "Mountain or Wilderness Rescue": [
        "German Shepherd", "Siberian Husky", "Alaskan Malamute", "Bernese Mountain Dog",
        "St. Bernard", "Border Collie", "Australian Shepherd", "Australian Cattle Dog"
    ],
    "Disaster or Individual Tracking": [
        "German Shepherd", "Doberman", "Rottweiler", "Bloodhound", "Beagle",
        "Coonhound", "Bluetick Coonhound", "Black and Tan Coonhound",
        "Treeing Walker Coonhound", "Plott Hound", "Redbone Coonhound",
        "American Foxhound", "Belgian Malinois"
    ],
}

# columns I expect to exist in the table and charts
TABLE_COLS = [
    "name","animal_type","breed","sex_upon_outcome",
    "age_upon_outcome_in_weeks","outcome_type","location_lat","location_long"
]

def build_query(rescue_type: str | None, max_age_weeks: int = 104) -> dict:
    """Dog-only, <= max_age_weeks, and breed list if a rescue_type is selected."""
    q = {
        "animal_type": {"$regex": "^Dog$", "$options": "i"},
        "age_upon_outcome_in_weeks": {"$lte": int(max_age_weeks)}
    }
    if rescue_type and rescue_type in RESCUE_BREEDS:
        q["$or"] = [{"breed": {"$regex": b, "$options": "i"}} for b in RESCUE_BREEDS[rescue_type]]
    return q

def load_df(rescue_type=None, max_age_weeks=104, limit=500):
    # protect notebook UX if the DB read throws (auth/connection)
    try:
        data = db.read(build_query(rescue_type, max_age_weeks), limit=limit)
    except Exception as e:
        # return an empty frame with expected columns so the UI still renders
        return pd.DataFrame(columns=TABLE_COLS)

    df = pd.DataFrame(data)
    if df.empty:
        return pd.DataFrame(columns=TABLE_COLS)

    # make sure these columns exist
    for c in TABLE_COLS:
        if c not in df.columns:
            df[c] = None

    # enforce numeric map coords
    df["location_lat"]  = pd.to_numeric(df["location_lat"], errors="coerce")
    df["location_long"] = pd.to_numeric(df["location_long"], errors="coerce")
    return df

# initial unfiltered
df = load_df(rescue_type=None, max_age_weeks=104)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

logo_path = "Grazioso Salvare Logo.png"
logo_tag = []
if os.path.exists(logo_path):
    encoded = base64.b64encode(open(logo_path, "rb").read()).decode()
    logo_tag = [html.Img(src=f"data:image/png;base64,{encoded}", style={"height": "50px", "marginRight": "12px"})]

app.layout = html.Div([
    html.Div(logo_tag + [
        html.H2("Grazioso Salvare — Rescue Candidate Dashboard", style={"margin":"0"}),
        html.P("Unique ID: Julliane Pamfilo — CS 340 Project Two", style={"margin":"0"})
    ], style={"display":"flex","alignItems":"center","gap":"12px"}),
    html.Hr(),

    # Controls
    html.Div([
        html.Div([
            html.Label("Rescue Type"),
            dcc.Dropdown(
                id="filter-type",
                options=[{"label": k, "value": k} for k in RESCUE_BREEDS.keys()],
                placeholder="Select a rescue type…",
                clearable=True,
            ),
        ], style={"minWidth":"260px","marginRight":"16px"}),

        html.Div([
            html.Label("Max age (weeks)"),
            dcc.Slider(
                id="max-age",
                min=12, max=156, step=4, value=104,
                marks={12:"12", 52:"52", 104:"104", 156:"156"},
                tooltip={"placement":"bottom","always_visible":False}
            ),
        ], style={"flex":1,"marginRight":"16px"}),

        html.Button("Reset", id="reset-btn", n_clicks=0)
    ], style={"display":"flex","alignItems":"center"}),

    html.Hr(),

    # Data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": c, "id": c} for c in df.columns] if not df.empty else [{"name": c, "id": c} for c in TABLE_COLS],
        data=df.to_dict('records'),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        style_table={"overflowX":"auto"},
        style_cell={"textAlign":"left","padding":"6px"},
    ),

    html.Br(),
    html.Hr(),

    # charts side-by-side
    html.Div(className='row', style={'display':'flex','gap':'16px'}, children=[
        html.Div(id='graph-id', className='col s12 m6', style={"flex":"1"}),
        html.Div(id='map-id',   className='col s12 m6', style={"flex":"1"})
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Update DataTable on filter changes or reset
@app.callback(
    Output('datatable-id','data'),
    Output('datatable-id','columns'),
    Output('datatable-id','selected_rows'),
    Input('filter-type','value'),
    Input('max-age','value'),
    Input('reset-btn','n_clicks'),
    prevent_initial_call=False
)
def update_dashboard(filter_type, max_age, n_clicks):
    # Dash 2.14+ exposes dash.ctx; older versions use dash.callback_context
    ctx = getattr(dash, "ctx", None) or dash.callback_context
    if ctx.triggered and "reset-btn" in (ctx.triggered[0].get("prop_id") or ""):
        dff = load_df(rescue_type=None, max_age_weeks=104)
    else:
        dff = load_df(rescue_type=filter_type, max_age_weeks=max_age)

    if dff.empty:
        cols = [{"name": c, "id": c} for c in TABLE_COLS]
        return [], cols, []
    else:
        cols = [{"name": c, "id": c} for c in dff.columns]
        return dff.to_dict("records"), cols, []

# Breed distribution bar chart based on current table data
@app.callback(
    Output('graph-id', "children"),
    Input('datatable-id', "data")
)
def update_graphs(rows):
    dff = pd.DataFrame(rows)
    if dff.empty or "breed" not in dff.columns:
        fig = px.bar(pd.DataFrame({"breed": [], "count": []}), x="breed", y="count", title="Top Breeds")
    else:
        top = (dff["breed"].fillna("Unknown")
               .str.split("/", n=1).str[0].str.strip()
               .value_counts().head(15)
               .rename_axis("breed").reset_index(name="count"))
        fig = px.bar(top, x="breed", y="count", title="Top Breeds in Current Filter")
        fig.update_layout(xaxis_tickangle=-35, margin=dict(l=10, r=10, t=40, b=120))
    return [dcc.Graph(figure=fig)]

# Highlight selected column(s) (kept from your template)
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    Input('datatable-id', 'selected_columns')
)
def update_styles(selected_columns):
    return [{'if': {'column_id': i}, 'background_color': '#D2F3FF'} for i in (selected_columns or [])]

# Map: show marker at selected row
@app.callback(
    Output('map-id', "children"),
    Input('datatable-id', "data"),
    Input('datatable-id', "derived_virtual_selected_rows")
)
def update_map(rows, sel):
    dff = pd.DataFrame(rows)
    if dff.empty:
        return [dl.Map(style={'width':'100%','height':'450px'}, center=[30.27,-97.74], zoom=10, children=[dl.TileLayer()])]

    # single row selection; default to first row if none selected
    row_idx = (sel or [0])[0]
    row_idx = min(max(row_idx, 0), len(dff)-1)

    lat = pd.to_numeric(dff.iloc[row_idx].get("location_lat"), errors="coerce")
    lon = pd.to_numeric(dff.iloc[row_idx].get("location_long"), errors="coerce")
    if pd.isna(lat) or pd.isna(lon):
        lat, lon = 30.27, -97.74

    name  = dff.iloc[row_idx].get("name")  or "Unknown"
    breed = dff.iloc[row_idx].get("breed") or "Unknown"

    return [
        dl.Map(style={'width':'100%','height':'450px'}, center=[float(lat), float(lon)], zoom=11, children=[
            dl.TileLayer(),
            dl.Marker(position=[float(lat), float(lon)], children=[
                dl.Tooltip(breed),
                dl.Popup([html.H4("Animal Name"), html.P(name)])
            ])
        ])
    ]

# Run (change port if needed)
app.run_server(mode="inline", port=8050, dev_tools_ui=False, dev_tools_props_check=False)